### 一. NIO Pipeline
1. java nio的pipeline包括非阻塞的读和非阻塞的写, 一个简化的过程是:
    * `Selector`检查哪个`Channel`有数据可读
    * 将数据从Channel中读出, 并根据输入产生输出
    * 将输出再次写回`Channel`
    
    
2. nio pipeline与io pipeline的区别   
   nio pipeline与io pipeline的最大区别在于: 数据如何从底层`channel`中读取.  
    1. 标准IO: 
        * 标准IO的读操作, 可以概括为从一些stream中读数据, 然后切分(split)数据成不同的message  
          这有些像使用tockenizer来解析字符串. 当有一个字节可被读取时, 阻塞版IO使用`InputStream`接口从底层的`channel`中读数据. 当使用`BufferedReader`包装`InputStream`后, 相当于使用数组暂存读取的消息, 然后使用换行符解析出每行数据   
        * 阻塞版IO, 大大简化了读操作的处理; 因为由于stream时阻塞的, 也就不需要处理stream无数据可读的情况, 使用BufferedReader包装输入流后, 也不需要担心partial message的暂存和完整消息到来后的延迟处理问题. 同样的, 阻塞版IO的写操作也不必担心只有部分响应写回, 还有剩余数据需要下次继续写回的情况. 
        * 阻塞版IO pipeline的弊端:
            1. 1个线程除以1个客户端  
              阻塞版IO需要对每个连接上的服务器开启一个线程去响应. 如果有100万个client连接过来, 就会开启100万个线程去执行响应. 由于每个线程占用1024k内存(64位JVM)或320k内存(32位内存), 1百万个线程就需要1T的内存, 这还仅仅是线程创建出来所需要的内存, 还不算处理数据需要的内存.为了减小线程数量, 很多server使用线程池处理连接的客户端, 第三这要求接入的客户端频繁发送消息, 如果线程池中包含了大量不活动的连接, 就仍会导致大量线程阻塞无法处理下一个接入socket, 就意味着server响应缓慢. 为解决这个问题, 一些server采用CachedThreadPool来处理这个问题, 但这仍然有一个处理上线的问题, 比如还是无法解决100万个客户端需要100万个线程处理的情况.   
    2. NIO
        * 非阻塞IO使用单线程从多个stream中读取数据  
           首先, stream要开启no-blocking模式, 该模式下, stream可能返回0个或多个字节; 返回0字节时表示没有数据可被读取, 返回多个字节时, 表示有一些数据可被读取. 为了避免持续检查stream是否返回了0字节,于是使用`Selector`. 1个或多个`SelectableChannel`实例被注册到了`Selector`上, 当使用`Selector`的`select()`或`selectNow()`方法时只会返回有数据可读的`SelectableChannel`实例

### 二. NIO诞生的新问题
1. **reading partial message**  
    当从`SelectableChannel`中读取数据块时,如果message代表1条逻辑上完整的消息, 则并不知道这个数据块的字节数是比1条message多还是少.   
    一个数据块可能是:  
    * partial message (小于一条message)
    * full message (完整的message)
    * 多余一条message  
    
   处理partial message有两个挑战:
    1. 检查数据块中是否包含一个完整message(full message)  
    2. 对于partial message, 直到剩余数据到来之前要做些什么?
    
   检查full message的程序, 当发现读到的数据块中包含一条或多条full message后, 这些full message就要被送往pipeline进行处理. 检查full mesage的程序要反复执行, 依次该检查程序的性能要尽可能的快;  对于partial message, 无论是数据块本身就是partial message, 还是在解析full message后剩余了partial message, 都要先缓存起来等待剩余的数据从`channel`中到来. 这两个功能都应该由`MessageReader`完成, 为避免不同的`Channel`之间的数据相互混淆, 我们给每个`channel`一个`MessageReader`实例. `Message Reader`需要满足特定协议, 来知道消息格式  
    <img src="img/non-blocking-server-6.png" width="48%" height="48%" >

2. **storing partial message**   
   现在, 我们已经知道要用`Message Reader`存储partial message直到剩余消息到来.关于这个问题有2个设计上的考虑: 
    1. 尽可能少的拷贝数据, 拷贝得越多性能越低
    2. 要把full message存储在一个连续的byte序列中, 这会使得解析message变得简单    
    
   * **每个MessageReader包含一个Buffer**  
   &nbsp;&nbsp;&nbsp;&nbsp;我们已经知道, 要给每个channel赋予一个`Message Reader`, 且每个`Message Reader`都要有1个内部buffer处理`channel`的数据块, 那应该使用多大的内部buffer呢? 首先要确保即使是最大的messgae也能被内部buffer存储; 其次对于不同的channel这个内部buffer的大小应该是不同的: 因为不同的channel可读取的数据不一样的, 可读数据较少的channel对应的MessageReader中的buffer也应该较小, 这样可以避免空间浪费. 比如若统一使用1MB的内部buffer, 100万个客户端连接共需要开启100万\*1MB=1TB的buffer.所以`Message Reader`内部的buffer应该是可变大小的(resizabel) 
   
   * **Resizebel Buffer**  
     有两种实现resizebale buffer的思路, 它们各有优缺点:
        1. resize by copy   
           第一种实现resizbale buffer的方法是: 先初始化一个小的buffer存储partial message, 比如4kb, 其余数据到来后若大于4kb, 则这4kb的partial message会被复制到一个8kb的buffer中
           * 这种方法的优点是所有消息都存储在一个连续的数组中, 后续解析message会很容易. 缺点是message越大, 复制过程中产生的中间数据越大 
           * 一旦1个full message被处理完成后, 对其分配的内存应该被释放, 下一次该连接上读到的数据会从最小的buffer size开始. 这很有必要, 为了让不同连接之间的内存共享更高效. 多数情况下, 同一时间不会是所有的连接都需要大容量的buffer.  
           
        2. resize by append  
           另一种实现思路是让buffer由多个数组组成, resizeble的方法就是分配其他的比特数组来存储读到的数据. 由2个方法来实现这种append:  
           1. 让buffer持有一个元素为数组的列表. `List<Object[]>` 
           2. 提供一个差大的,被不同buffer共享的数组, buffer中记录着共享数组的切片位置. `List<(startIndex,endIndex)>`     
          
   * **TLV encoded message**  
     `Message Reader`要满足某种协议的, 这种协议就是一个消息格式, 包含3部分:  
        1. Type
        2. Length
        3. Value   
    
    这表示, 当一条消息的长度会存储在消息的开头, 当一条消息到来后可以立刻知道启动多大的buffer存储这个消息.   
    TLV encoding使内存管理变得容易, 你可以立刻知道分配多大的内存来存储一条消息. 但是TLV也有一个缺点, 就是要在消息完整到达以前对该消息分配完整的数组大小. 一些缓慢却要发送很答消息的连接会拖慢server端的表现. 解决这个问题有2个思路:  
        1. 条消息包含多个TLV encoding的字段   
         此时, 内存分配并不是针对一条完整消息, 而是针对每个字段. 但是带有大量数据的字段仍然会带来相同的问题
        2. 对消息赋予一个超时时间  
         不如我们可以设定, 如果15秒后还没收到一条完整的消息就清空已分配的内存    
         
3. **writing partial message**  
   &nbsp;&nbsp;&nbsp;&nbsp;非阻塞版IO写回数据是一个挑战, 当在非阻塞模式下调用`channel`的`write(ByteBuffer)`方法时, 并不能保证`ByteBuffer`中有多少数据被写回; `write(ByteBuffer)`方法返回写回的字节数. 因此, 这个挑战是"监控写回了partial message,使得最终全部消息被发送"   
   &nbsp;&nbsp;&nbsp;&nbsp;partial message的写回应该由`Message Writer`处理, 和`Message Reader`一样, 每个`channel`拥有自己的`Message Writer`, 负责跟踪本次写回共写回了多少字节的数据. 因为可能有多个完整的message需要被写回, 所以`Message Writer`中要有一个队列存储待写回`channel`的message. 为了避免轮训检查哪个`channel`处于可写状态, 可以使用如下两步进行数据发送:
    1. 当有数据写到`Message Writer`后, 把`channel`的`写SelectionKey`注册到`selector`  
    2. `Selector`发现`channel`可写后返回, `Message Writer`将数据写回`channel`, 当所有的消息都被完整写回后将`channel`的`写SelectionKey`从`selector`中解除绑定  
    
   如上两步, 就能保证只有有数据待写回的`channel`才会被注册到`selector`上  